<a href="https://colab.research.google.com/github/WASasquatch/openai-guided-diffusion-colab/blob/main/OpenAI_Guided_Diffusion_Model_Training_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenAI Guided Diffusion - Model Training / Fine-Tuning ![visitors](https://visitor-badge.glitch.me/badge?page_id=OponAI-Guided-Diffusion-Colab&left_color=blue&right_color=orange)

A user friendly (hopefully) notebook for training Diffusion Models for CLIP Guided Diffusion on [Disco Diffusion](https://discodiffusion.com/). This notebook is compatible with T4/P100 GPUs *(when batch size is set to ~2)*.

Notebook created by **WAS**asquatch <font size="2">(<font color="7e87f6">Discord</font>: WAS\#0263)</font>

[*OpenAI Guided Diffusion*](https://github.com/openai/guided-diffusion) *is licensed under MIT license*

## Other Tools

- [Noodle Soup Prompts](https://rebrand.ly/noodle-soup-prompts) - Terminology Database for prompt exploration
- [Portrait Playground](https://rebrand.ly/portrait-playground) - A version of Disco Diffusion with all current Portrait Generator Models. 
- [Portrait Generator `init_image` Prep](https://rebrand.ly/dd-image-prep) - Prep portraits other other `init_images` as approximations for better DD improvisation. 

# Setup Environment

In [ ]:
#@title 1) Check GPU Status
import subprocess

simple_nvidia_smi_display = False#@param {type:"boolean"}

if simple_nvidia_smi_display:
  #!nvidia-smi
  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
else:
  #!nvidia-smi -i 0 -e 0
  nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
  nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_ecc_note)

nvidiasmi_simple = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
gpu_name = nvidiasmi_simple.split(':')[1].split('(')[0].strip()

gpu_warning = None
if 'Tesla T4' or 'Tesla P100' in gpu_name:
  gpu_warning = "\033[1m\33[31mWARNING:\33[0m \033[1mYou are using a limited VRAM GPU (\33[33m"+gpu_name+"\33[0m\033[1m). You'll need to change \"_batch_size\" to 2 or under.\33[0m"
  print(gpu_warning)

In [ ]:
#@title 2) Install Pre-requisites and Scripts
import IPython, psutil, os, time
from google.colab import output
from IPython.display import clear_output 

class log:
    f = lambda color: lambda string: print(color + string + "\33[0m")
    black = f("\33[30m")
    red = f("\33[31m")
    green = f("\33[32m")
    yellow = f("\33[33m")
    blue = f("\33[34m")
    megenta = f("\33[35m")
    cyan = f("\33[36m")
    white = f("\33[37m")
    bold = f("\033[1m")
    boldred = f("\033[1m\33[31m")
    boldgreen = f("\033[1m\33[32m")
    boldyellow = f("\033[1m\33[33m")
    boldblue = f("\033[1m\33[34m")
    boldmegenta = f("\033[1m\33[35m")
    boldcyan = f("\033[1m\33[36m")

from google.colab import drive
drive.mount('/content/drive')

%cd /content
!git clone https://github.com/openai/guided-diffusion
%cd /content/guided-diffusion  
!pip install -e .
!pip install mpi4py 

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

log.boldgreen("Pre-Requisites Installed.")

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()
log.yellow("Time until session disconnect: "+str(round((43200-uptime)/3600))+" hours")


# Train Diffusion Model



## Path Settings:

 - <font size="5">`_dataset_path` = Path to your dataset folder containing images.</font>
 - <font size="5">`_checkpoint_path` = Path to a folder where Guided Diffusion will save progress checkpoints.</font>
 - <font size="5">`_resume_checkpoint_path` = Path to a diffusion model to resume training/fine-tuning





Because this is Guided Diffusion, CLIP will help build your images. This means you do not need to spend as much time training your model. Though, a fair amount of training is still required for a cohesive return. 

The notebook will train until your GPU quota is up, you stop it, or there is an error. You can check your progress by loading the `ema_0.9999_[num_steps_trained].pt` file as a Custom Model in [Disco Diffusion](https://discodiffusion.com/)

Your custom model will require you to change the Custom Model Settings in order to run it. Probably something like like the following (though if you change trainning settings it may differ)

## Custom Model Settings (Example)

<font size="3">

```
  model_config.update({
      'attention_resolutions': '32, 16, 8',
      'class_cond': False,
      'diffusion_steps': 1000,
      'rescale_timesteps': True,
      'image_size': 512,
      'learn_sigma': True,
      'noise_schedule': 'linear',
      'num_channels': 128,
      'num_heads': 4,
      'num_res_blocks': 2,
      'resblock_updown': True,
      'use_checkpoint': use_checkpoint,
      'use_fp16': True,
      'use_scale_shift_norm': True,
    })
```
</font>

# Training Settings

In [ ]:
#@markdown ## **Basic Settings**
#@markdown ---
_dataset_path = "/content/drive/MyDrive/AI/Datasets/set1/" #@param{type: 'string'}
_checkpoint_path = "/content/drive/MyDrive/AI/Checkpoints/concept-art-model/" #@param{type: 'string'}
_resume_checkpoint_path = "/content/drive/MyDrive/AI/Checkpoints/concept-art-model/ema_0.9999_070000.pt" #@param{type:'string'}
resume_checkpoint = True #@param{type: 'boolean'}
#@markdown &nbsp;&nbsp;&nbsp; <font color="orange">**NOTE**:</font> This resumes training a model specified in `_resume_checkpoint_path`

_diffusion_steps = 1000 #@param{type: 'number'}
_image_size = 512 #@param{type: 'number'}

#@markdown &nbsp;
#@markdown ## **Advanced Settings**
#@markdown <br >

#@markdown #### **Model Flags**
_attention_resolutions = "32,16,8" #@param{type: 'string'}
_class_cond = False #@param{type: 'boolean'}
_learn_sigma = True #@param{type: 'boolean'}
_noise_schedule = "linear" #@param{type: 'string'}
_num_channels = 128 #@param{type: 'number'}
_num_heads = 4 #@param{type: 'number'}
_num_res_blocks = 2 #@param{type: 'number'}
_resblock_updown = True #@param{type: 'boolean'}
_use_fp16 = True #@param{type: 'boolean'}
_use_scale_shift_norm = True #@param{type: 'boolean'}
custom_model_flags = "" #@param{type: 'string'}
#@markdown &nbsp;&nbsp;&nbsp; Customize flags which are not part of the settings.

#@markdown &nbsp;
#@markdown #### **Training Flags**
_lr = 2e-5 #@param{type: 'string'}
_batch_size = 2 #@param{type: 'number'}
#@markdown &nbsp;&nbsp;&nbsp; <font color="orange">**NOTE**:</font> It's important to set this according to GPU VRAM size. For Colab Free/Pro on T4/P100 a `_batch_size` of `2` will work.
_save_interval = 10000 #@param{type: 'number'}
#@markdown &nbsp;&nbsp;&nbsp; <font color="orange">**NOTE**:</font> Save interval is in ***steps***.
_log_interval = 50 #@param{type: 'number'}
#@markdown &nbsp;&nbsp;&nbsp; <font color="orange">**NOTE**:</font> Log interval is in ***steps***.
custom_training_flags = "" #@param{type: 'string'}
#@markdown &nbsp;&nbsp;&nbsp; Customize flags which are not part of the settings.

#@markdown &nbsp;
#@markdown #### **Export Settings**
_use_checkpoint = True #@param{type: 'boolean'}
_use_dd_checkpoint = True #@param{type: 'boolean'}
#@markdown &nbsp;&nbsp;&nbsp; Use Disco Diffusion `use_checkpoint` variable.

_use_dd_checkpoint = 'use_checkpoint' if _use_dd_checkpoint else _use_checkpoint

log.boldgreen("SETTINGS COMPILED.")
print("")
log.boldblue("CUSTOM MODEL SETTINGS:")
print("")

log.bold("  model_config.update({")
log.bold("      'attention_resolutions': '"+_attention_resolutions+"',")
log.bold("      'class_cond': "+str(_class_cond)+",")
log.bold("      'diffusion_steps': "+str(_diffusion_steps)+",")
log.bold("      'rescale_timesteps': True,")
log.bold("      'image_size': "+str(_image_size)+",")
log.bold("      'learn_sigma': "+str(_learn_sigma)+",")
log.bold("      'noise_schedule': '"+str(_noise_schedule)+"',")
log.bold("      'num_channels': "+str(_num_channels)+",")
log.bold("      'num_heads': "+str(_num_heads)+",")
log.bold("      'num_res_blocks': "+str(_num_res_blocks)+",")
log.bold("      'resblock_updown': "+str(_resblock_updown)+",")
log.bold("      'use_checkpoint': "+str(_use_dd_checkpoint)+",")
log.bold("      'use_fp16': "+str(_use_fp16)+",")
log.bold("      'use_scale_shift_norm': "+str(_use_scale_shift_norm)+",")
log.bold("    })")

if gpu_warning:
  print("")
  print(gpu_warning)

# 3) Train Guided Diffusion Model

In [ ]:
#@title Start Guided Diffusion training / fine-tuning
#@markdown Run this cell to start the training session. 
_resume_checkpoint = '--resume_checkpoint ' if resume_checkpoint else ''
_resume_checkpoint_path = _resume_checkpoint_path if resume_checkpoint else ''

MODEL_FLAGS = f"--attention_resolutions {_attention_resolutions} --class_cond {_class_cond} --diffusion_steps {_diffusion_steps} --image_size {_image_size} --learn_sigma {_learn_sigma} --noise_schedule {_noise_schedule} --num_channels {_num_channels} --num_heads {_num_heads}  --num_res_blocks {_num_res_blocks} --resblock_updown {_resblock_updown} --use_fp16 {_use_fp16} --use_scale_shift_norm {_use_scale_shift_norm} {custom_model_flags}"
TRAIN_FLAGS = f"--lr {_lr} --batch_size {_batch_size} --save_interval {_save_interval} --log_interval {_log_interval} {_resume_checkpoint}{_resume_checkpoint_path} {custom_training_flags}" 
DATASET_PATH=_dataset_path
CHECKPOINT_PATH=_checkpoint_path

%cd /content/guided-diffusion/
!OPENAI_LOGDIR=$CHECKPOINT_PATH python scripts/image_train.py --data_dir $DATASET_PATH $MODEL_FLAGS $TRAIN_FLAGS